In [1]:
import numpy as np
import pandas as pd
from sklearn import svm
import pickle as pk

In [2]:
X_np = np.load("X_trn.np")
Y_np = np.load("Y_trn.np")

In [3]:
print(X_np.shape)
print(Y_np.shape)

(163, 1600)
(163,)


In [4]:
def cleanclicks(xtrn, ytrn):
    x_trn = []
    inds_bad = []
    for i in xrange(len(xtrn)):
        if len(xtrn[i]) < 1600:
            inds_bad.append(i)
        x_trn.append(list(xtrn[i]))
    for i in sorted(inds_bad, reverse=True):
        del x_trn[i]
    y_trn = np.delete(ytrn, inds_bad, 0)
    x_trn = np.array(x_trn)
    return x_trn, y_trn

X_np, Y_np = cleanclicks(X_np, Y_np)

In [5]:
print(X_np.shape)
print(Y_np.shape)

(163, 1600)
(163,)


In [6]:
X_df = pd.DataFrame(X_np)
X_df.head()

,0,1,2,3,4,5,6,7,8,9,...,1590,1591,1592,1593,1594,1595,1596,1597,1598,1599
0,0.482353,0.505882,0.498039,1,0.474510,0.498039,0.490196,1,0.411765,0.450980,...,0.125490,1,0.011765,0.043137,0.000000,1,0.133333,0.164706,0.113725,1
1,0.592157,0.572549,0.498039,1,0.501961,0.482353,0.407843,1,0.192157,0.168627,...,0.023529,1,0.082353,0.003922,0.000000,1,0.082353,0.003922,0.000000,1
2,0.156863,0.156863,0.156863,1,0.278431,0.286275,0.282353,1,0.207843,0.215686,...,0.392157,1,0.423529,0.439216,0.372549,1,0.403922,0.419608,0.352941,1
3,0.568627,0.619608,0.549020,1,0.658824,0.690196,0.631373,1,0.682353,0.713726,...,0.878431,1,0.894118,0.909804,0.850980,1,0.313726,0.341176,0.278431,1
4,0.062745,0.070588,0.058824,1,0.050980,0.050980,0.043137,1,0.047059,0.047059,...,0.294118,1,0.294118,0.301961,0.282353,1,0.270588,0.290196,0.266667,1


In [7]:
Y_np[0]

1.0

In [53]:
# probablity, likliehood, confidence interval, gaussian, logistic, multi-class logistic regression, discrete choice
# neural network
# random forest
# decision tree - can only sepereate on each feature of the vector
# C big C overfit the data; find best C with cross-validation
# if sigma is large, then gamma is small.... if sigma is large then reach lots of points (meaing you are underfitting)
# gamma specifically for rbf kernel

clf = svm.SVC(kernel = 'rbf', C=1.0, gamma=0.01)
clf.fit(X_np, Y_np)

with open('classifier_CarloLiquido.pickle','wb') as f:
    pk.dump(clf, f)

In [54]:
# cleaning function on piazza that helps take out features less than 1600... might be broken so take that out; will give
# numpy arrawys
# 

In [55]:
preview = np.load("test_locations_and_labels.np")

In [56]:
pd.DataFrame(preview)

,0,1,2
0,271,493,1
1,323,481,1
2,560,475,1
3,576,524,1
4,287,553,1
5,68,483,0
6,168,479,0
7,374,502,0
8,332,559,0
9,637,541,0


** experiment using crossvalidation **

In [18]:
from sklearn import cross_validation 
X_train, X_test, y_train, y_test = cross_validation.train_test_split(
    X_np, Y_np, test_size=0.4, random_state=0)

In [19]:
print(X_train.shape, y_train.shape)

print(X_test.shape, y_test.shape)  

((97, 1600), (97,))
((66, 1600), (66,))


In [23]:
clf_experiment = svm.SVC(kernel = 'rbf', C=10.0).fit(X_train, y_train)

clf.score(X_test, y_test) 

1.0

** experiment2 using crossvalidation with k-folds **

In [28]:
clf_expirement2 = svm.SVC(kernel = 'rbf', C=90.0)
scores = cross_validation.cross_val_score(
    clf_expirement2, X_np, Y_np, cv=5)
scores

array([ 0.84848485,  0.78787879,  0.78787879,  0.875     ,  0.90625   ])

In [29]:
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

Accuracy: 0.84 (+/- 0.09)


** crossvalidation3 with gridsearch **

In [49]:
tuned_parameters = [{'kernel': ['rbf'],
                     'gamma': [1e-1, 1e-2, 1e-3, 1e-4], 'C': [1, 10, 50, 100, 500, 1000]}
                   ]

In [50]:
tuned_parameters

[{'C': [1, 10, 50, 100, 500, 1000],
  'gamma': [0.1, 0.01, 0.001, 0.0001],
  'kernel': ['rbf']}]

In [51]:
scores3 = ['precision', 'recall']

In [52]:
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report

for score in scores3:
    print("# Tuning hyper-parameters for %s" % score)
    print()

    clf = GridSearchCV(svm.SVC(C=1), tuned_parameters, cv=5,
                       scoring='%s_weighted' % score)
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    for params, mean_score, scores in clf.grid_scores_:
        print("%0.3f (+/-%0.03f) for %r"
              % (mean_score, scores.std() * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for precision
()
Best parameters set found on development set:
()
{'kernel': 'rbf', 'C': 1, 'gamma': 0.01}
()
Grid scores on development set:
()
0.905 (+/-0.073) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.1}
0.922 (+/-0.121) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.01}
0.449 (+/-0.045) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.001}
0.449 (+/-0.045) for {'kernel': 'rbf', 'C': 1, 'gamma': 0.0001}
0.905 (+/-0.073) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.1}
0.916 (+/-0.117) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.01}
0.838 (+/-0.145) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.001}
0.449 (+/-0.045) for {'kernel': 'rbf', 'C': 10, 'gamma': 0.0001}
0.905 (+/-0.073) for {'kernel': 'rbf', 'C': 50, 'gamma': 0.1}
0.916 (+/-0.117) for {'kernel': 'rbf', 'C': 50, 'gamma': 0.01}
0.897 (+/-0.075) for {'kernel': 'rbf', 'C': 50, 'gamma': 0.001}
0.761 (+/-0.159) for {'kernel': 'rbf', 'C': 50, 'gamma': 0.0001}
0.905 (+/-0.073) for {'kernel': 'rbf', 'C': 100, 'gamma': 0.1}
0.916 (+